In [39]:
from math import pi
from CoolProp.CoolProp import PropsSI

In [40]:
def circular_area(diameter):
    return diameter ** 2 * pi / 4


def outlet_enthalpy_isentropic_expansion(h_1, p_1, p_2, eta, wf):
    h_2s = PropsSI("H", "P", p_2, "S", PropsSI("S", "H", h_1, "P", p_1, wf), wf)
    return eta * (h_2s - h_1) + h_1


def sound_speed(p, h, wf):
    Q = PropsSI("Q", "P", p, "H", h, wf)
    if Q > 0 and Q < 1:
        c_l = PropsSI("A", "P", p, "Q", 0, wf)
        c_g = PropsSI("A", "P", p, "Q", 1, wf)
        rho_l = PropsSI("D", "P", p, "Q", 0, wf)
        rho_g = PropsSI("D", "P", p, "Q", 1, wf)

        return (
            1 / (
                (1 - Q) * ((1 - Q) / (c_l ** 2) + (Q * rho_l) / (rho_g * c_g ** 2)) ** 0.5
                + Q * ((Q / c_g ** 2) + ((1 - Q) * rho_g) / (rho_l * c_l ** 2)) ** 0.5
            )
        ) ** 0.5

    else:
        return PropsSI("A", "P", p, "H", h, wf)


D_t = 2.82 * 1e-3
D_p1 = 5.10 * 1e-3
D_3 = 9.20 * 1e-3
P_p = 6.04 * 1e5
P_s = 0.40 * 1e5
ω = 0.438
eta_n = 0.95

wf = "R141b"

A_t = circular_area(D_t)
A_p1 = circular_area(D_p1)
A_3 = circular_area(D_3)

# assume saturated states
T_p = PropsSI("T", "P", P_p, "Q", 0, wf)
T_s = PropsSI("T", "P", P_s, "Q", 1, wf)
h_p = PropsSI("H", "P", P_p, "Q", 0, wf)
h_s = PropsSI("H", "P", P_s, "Q", 1, wf)

P_t = P_p * 0.8
d = 1e-3

while True:

    h_t = outlet_enthalpy_isentropic_expansion(h_p, P_p, P_t, eta_n, wf)
    c_t = sound_speed(P_t, h_t, wf)
    h_t_2 = h_p - c_t ** 2 / 2
    residual = h_t - h_t_2

    P_t += d

    h_t = outlet_enthalpy_isentropic_expansion(h_p, P_p, P_t, eta_n, wf)
    c_t = sound_speed(P_t, h_t, wf)
    h_t_2 = h_p - c_t ** 2 / 2
    residual_u = h_t - h_t_2

    derivative = (residual_u - residual) / (d)
    P_t -= d

    P_t -= residual / derivative
    if abs(residual) < 1e-6:
        break

h_t = outlet_enthalpy_isentropic_expansion(h_p, P_p, P_t, eta_n, wf)
rho_t = PropsSI("D", "P", P_t, "H", h_t, wf)
c_t = (2 * (h_p - h_t)) ** 0.5
m = rho_t * A_t * c_t
m


0.038209690024382

In [27]:
import numpy as np

P_p1 = P_t * np.linspace(0.1, 0.3)

# while True:
h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)
rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
c_p1 = m / (A_p1 * rho_p1)
h_p1_2 = h_t + c_t ** 2 / 2 - c_p1 ** 2 / 2

residual = h_p1 - h_p1_2

residual

array([28360.1333474 , 25186.17660751, 22400.02945359, 19944.36416638,
       17771.89558648, 15843.35592113, 14125.92886461, 12592.02795358,
       11218.33548957,  9985.04053365,  8875.23031447,  7874.40082646,
        6970.06073718,  6151.40886766,  5409.07007674,  4734.87780415,
        4121.69411396,  3563.26005014,  3054.07062642,  2589.26993928,
        2164.56279929,  1776.13998487,  1420.61478016,  1094.9688995 ,
         796.50625163,   522.81327717,   271.72481745,    41.29465496,
        -170.2299879 ,  -364.43058462,  -542.73657063,  -706.44215445,
        -856.72102311,  -994.63923598, -1121.16655627, -1237.18643134,
       -1343.50480165, -1440.85789189, -1529.91911551, -1611.30520526,
       -1685.58166662, -1753.26763763, -1814.84022717, -1870.73839429,
       -1921.36642269, -1967.0970375 , -2008.27420561, -2045.21565524,
       -2078.21514622, -2107.54451865])

In [28]:
P_p1 = P_t * 0.1

while True:
    h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)
    rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
    c_p1 = m / (A_p1 * rho_p1)
    h_p1_2 = h_t + c_t ** 2 / 2 - c_p1 ** 2 / 2

    residual = h_p1 - h_p1_2

    P_p1 += d

    h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)

    rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
    c_p1 = m / (A_p1 * rho_p1)
    h_p1_2 = h_t + c_t ** 2 / 2 - c_p1 ** 2 / 2

    P_p1 -= d

    derivative = ((h_p1 - h_p1_2) - residual) / d

    P_p1 -= residual / derivative

    if abs(residual) <= 1e-6:
        break


# recalculate to get true values
h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)
rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
c_p1 = m / (A_p1 * rho_p1)

P_p, P_t, P_p1

(604000.0, 491984.09806351893, 103795.83652814342)